- Named Entity Recognition (NER) is a critical sub-task of Information Extraction, focusing on identifying specific entities within the text and categorizing them into predefined classes such as names of people, organizations, locations, dates, and numerical values.
- The following sections will thoroughly explore NER, its methods, applications, and creative observations, including code demonstrations where relevant.



#### 5.1 **Definition and Objectives of Named Entity Recognition**
   - **Definition**:
     - Named Entity Recognition (NER) is an NLP task that identifies named entities in text and classifies them into categories such as PERSON, ORGANIZATION, LOCATION, DATE, MONEY, etc.
     - NER serves as a foundational step in many NLP applications like information extraction, question answering, and machine translation.

   - **Objectives**:
     - **Entity Detection**: Locate named entities in a given text.
     - **Classification**: Classify the identified entities into relevant categories.
     - **Structuring Textual Data**: Create structured representations by extracting key information from unstructured text.

   - **Example Code**:


In [ ]:
import spacy

# Load a pre-trained model from spaCy for Named Entity Recognition (NER)
nlp = spacy.load("en_core_web_sm")

# Define a sample text to analyze
text = "Apple was founded by Steve Jobs in Cupertino."

# Process the text with spaCy to create a Doc object
doc = nlp(text)

# Extract named entities and their categories
for ent in doc.ents:
    # Print the entity text and its corresponding label
    print(ent.text, ent.label_)



Apple ORG
Steve Jobs PERSON
Cupertino GPE


#### 5.2 **Types of Named Entities**
   - **Basic Entity Types**:
     - **PERSON**: Names of individuals (e.g., "Albert Einstein").
     - **ORGANIZATION**: Names of companies, institutions, or groups (e.g., "NASA").
     - **LOCATION**: Geographic locations such as cities, countries, or landmarks (e.g., "Mount Everest").
     - **DATE/TIME**: Specific dates or times (e.g., "January 1, 2023").

   - **Additional Entity Types**:
     - **MONEY**: Monetary values (e.g., "$100").
     - **PERCENT**: Percentage values (e.g., "50%").
     - **GPE (Geo-Political Entities)**: Countries, cities, states (e.g., "Germany").
     - **FACILITY**: Names of facilities such as buildings, airports (e.g., "Eiffel Tower").
     - **EVENT**: Major historical or cultural events (e.g., "World War II").

   - **Example Code**:


In [ ]:
# Demonstrating extraction of multiple entity types using spaCy
import spacy

# Load the pre-trained spaCy model for Named Entity Recognition (NER)
nlp = spacy.load("en_core_web_sm")

# Define a sample text to analyze
text = "Bill Gates donated $100 million to the Covid Relief Fund in 2021."

# Process the text with spaCy to create a Doc object
doc = nlp(text)

# Extract and print entities and their categories
for ent in doc.ents:
    print(ent.text, ent.label_)



Bill Gates PERSON
$100 million MONEY
the Covid Relief Fund ORG
2021 DATE


#### 5.3 **NER Techniques**


   - **5.3.1 Dictionary and Gazetteer-Based Approaches**:
     - **Description**:
       - Using precompiled lists of known entities (e.g., cities, country names).
       - Suitable for high precision in domains with a limited set of entity names.
     - **Limitations**:
       - Poor generalization for entities not in the dictionary.
       - Inability to handle dynamic, evolving data (e.g., emerging person names).
     - **Example Code**:


In [ ]:
# Define known entities and their types in a dictionary
known_entities = {"New York": "GPE", "Elon Musk": "PERSON", "SpaceX": "ORG"}

# Define a sample text to analyze
text = "Elon Musk launched SpaceX in New York."

# Iterate through each word in the text
for word in text.split():
    # Check if the word is a known entity
    if word in known_entities:
        # Print the entity and its type
        print(f"Entity: {word}, Type: {known_entities[word]}")


Entity: SpaceX, Type: ORG


   - **5.3.2 Rule-Based Approaches**:
     - **Description**:
       - Use of linguistic patterns to identify named entities.
       - Example: Identifying dates using patterns like "DD-MM-YYYY" or "Month Year".
     - **Advantages**:
       - High interpretability and easy to modify.
       - Suitable for domains with specific, predictable patterns.
     - **Limitations**:
       - Rule-based approaches require manual effort and struggle with unseen entity formats.
     - **Example Code**:


In [ ]:
import re

# Regular expression to find dates in text
# The pattern looks for:
# - One or two digits for the day (\d{1,2})
# - A space followed by a month name (e.g., January, February, etc.)
# - A space followed by four digits for the year (\d{4})
date_pattern = r"\b\d{1,2}\s(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}\b"

# Sample text that includes a date
text = "The event will take place on 15 September 2023."

# Use re.findall() to find all occurrences of dates in the text
dates = re.findall(date_pattern, text)

# Print the dates found in the text
print("Dates found:", dates)



Dates found: ['15 September 2023']


   - **5.3.3 Supervised Learning Approaches**:
     - **Description**:
       - Training machine learning models on labeled corpora to recognize and classify entities.
       - Examples include algorithms like Conditional Random Fields (CRF) and decision trees.
     - **Datasets**:
       - Use labeled datasets such as CoNLL-2003 for training and evaluation.
     - **Example Code**:


In [ ]:
# Simple demonstration using sklearn for named entity classification
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np  # Import numpy

# Sample training data: sentences with named entities
# **Change:** Added a sentence without the target entities to ensure both classes (0 and 1) are present
X = ["Steve Jobs founded Apple", "Bill Gates founded Microsoft", "The weather is nice today"]
# Labels indicating the type of named entities present in each sentence
# **Change:** y is now a list of lists, with each sublist containing labels for the corresponding sentence
y = [["PERSON", "ORG"], ["PERSON", "ORG"], []]  # Empty list for the sentence without target entities

# Vectorize the training sentences
# Convert text data to a matrix of token counts
vectorizer = CountVectorizer()
X_vect = vectorizer.fit_transform(X)

# **Change:** Instead of reshaping y, we'll train a separate model for each entity type
# This approach avoids the inconsistent number of samples issue
models = {}  # Dictionary to store models for each entity type
for entity_type in set(entity for sublist in y for entity in sublist):
    # Create a binary target variable for the current entity type
    y_binary = [[1 if entity_type in sublist else 0] for sublist in y]
    # Reshape y_binary to be a 1D array
    y_binary = np.array(y_binary).ravel()
    # Train a model for the current entity type
    model = LogisticRegression()
    model.fit(X_vect, y_binary)
    models[entity_type] = model

# Test the model on a new sentence
test_sentence = ["Elon Musk founded Tesla"]
# Vectorize the test sentence
test_vect = vectorizer.transform(test_sentence)

# Predict the entity types in the test sentence
predictions = {}
for entity_type, model in models.items():
    prediction = model.predict(test_vect)
    if prediction[0] == 1:  # If the model predicts the entity type is present
        predictions[entity_type] = test_sentence[0].split()[prediction.tolist().index(1) if 1 in prediction.tolist() else -1]  # Assign the corresponding word


# Output the predicted entities for the test sentence
print("Predicted entities:", predictions)

Predicted entities: {'PERSON': 'Elon', 'ORG': 'Elon'}


   - **5.3.4 Neural Network-Based Approaches**:
     - **Deep Learning Models**:
       - **BiLSTM-CRF**: Combines bidirectional LSTMs for contextual understanding and CRF for sequence labeling.
       - **Transformers (e.g., BERT)**: Utilizes self-attention mechanisms for understanding word dependencies in context.
     - **Advantages**:
       - State-of-the-art performance with high accuracy due to deep contextual understanding.
       - Can generalize well with the availability of large datasets.
     - **Example Code**:


In [ ]:
from transformers import pipeline

# Using a pre-trained model for Named Entity Recognition (NER)
# The model is specifically fine-tuned for recognizing named entities on the CoNLL-03 dataset
nlp = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

# Define a sample text for named entity recognition
text = "Barack Obama was born in Hawaii and served as the 44th president of the United States."

# Use the NER pipeline to analyze the text
result = nlp(text)

# Print the detected entities with their details
for entity in result:
    print(entity)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'entity': 'I-PER', 'score': 0.99897027, 'index': 1, 'word': 'Barack', 'start': 0, 'end': 6}
{'entity': 'I-PER', 'score': 0.9994301, 'index': 2, 'word': 'Obama', 'start': 7, 'end': 12}
{'entity': 'I-LOC', 'score': 0.99943894, 'index': 6, 'word': 'Hawaii', 'start': 25, 'end': 31}
{'entity': 'I-LOC', 'score': 0.99924797, 'index': 16, 'word': 'United', 'start': 72, 'end': 78}
{'entity': 'I-LOC', 'score': 0.99812096, 'index': 17, 'word': 'States', 'start': 79, 'end': 85}


#### 5.4 **Evaluation of NER Systems**
   - **Evaluation Metrics**:
     - **Precision**: Ratio of correctly identified entities to the total entities predicted.
     - **Recall**: Ratio of correctly identified entities to the total number of true entities.
     - **F1-Score**: Harmonic mean of precision and recall, providing a balanced measure of performance.
   
   - **Cross-Validation**:
     - Evaluating the model using multiple folds of the dataset for robustness.
   
   - **Confusion Matrix**:
     - Useful in identifying specific types of errors (e.g., misclassification between PERSON and ORGANIZATION).
   
   - **Example Code**:


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# True and predicted entities for demonstration purposes
true_entities = ["PERSON", "ORG", "LOC", "PERSON"]  # Ground truth labels
predicted_entities = ["PERSON", "ORG", "LOC", "ORG"]  # Predicted labels by the model

# Calculate precision, recall, and F1-score using micro averaging
precision = precision_score(true_entities, predicted_entities, average="micro")
recall = recall_score(true_entities, predicted_entities, average="micro")
f1 = f1_score(true_entities, predicted_entities, average="micro")

# Print the evaluation metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)



Precision: 0.75
Recall: 0.75
F1-Score: 0.75


### 5.5 **Challenges in Named Entity Recognition**



#### 5.5.1 Ambiguity in Named Entity Recognition
- **Example**: Handling the ambiguity of "Apple" which can refer to both a fruit or a company, depending on the context.


In [ ]:
import spacy

# Load a spaCy pre-trained model for Named Entity Recognition
nlp = spacy.load("en_core_web_sm")

# Two sentences with ambiguous entities
text_1 = "Apple is a leading technology company."
text_2 = "She bought an apple from the store."

# Processing each text separately to demonstrate the difference in context
doc_1 = nlp(text_1)
doc_2 = nlp(text_2)

# Extracting and displaying named entities for the first text
for ent in doc_1.ents:
    # The model correctly identifies 'Apple' as an organization
    print(f"Text: '{text_1}', Entity: {ent.text}, Label: {ent.label_}")

# Extracting and displaying named entities for the second text
for ent in doc_2.ents:
    # In this context, the word 'apple' is a common noun and is not identified as a named entity
    print(f"Text: '{text_2}', Entity: {ent.text}, Label: {ent.label_}")

# Output:
# Text: 'Apple is a leading technology company.', Entity: Apple, Label: ORG


Text: 'Apple is a leading technology company.', Entity: Apple, Label: ORG


#### 5.5.2 Emerging Entities
- **Example**: Handling emerging entities, such as new celebrities or technologies that are not in the training data.


In [ ]:
# Example sentence with a new, emerging entity
text = "John Doe recently launched the new tech startup called InnovX."

# Using spaCy to process the text
doc = nlp(text)

# Displaying entities identified by the model
for ent in doc.ents:
    # The model might not correctly identify 'InnovX' since it is a new, emerging entity
    print(f"Entity: {ent.text}, Label: {ent.label_}")



Entity: John Doe, Label: PERSON


#### 5.5.3 Multi-Word Entities and Nested Entities
- **Example**: Identifying multi-word entities and nested entities such as "University of California, Berkeley."



In [ ]:
import spacy

# Load a spaCy pre-trained model for Named Entity Recognition (NER)
nlp = spacy.load("en_core_web_sm")

# Sentence with a multi-word entity and a nested entity
# 'University of California, Berkeley' is an example of a multi-word organization name
# 'United States' is another entity in the sentence, representing a location
text = "The University of California, Berkeley is located in the United States."

# Processing the text with spaCy to create a Doc object
# The Doc object contains linguistic annotations for the input text, such as entities, tokens, etc.
doc = nlp(text)

# Extracting named entities from the processed text
for ent in doc.ents:
    # Print each entity found in the text along with its label
    # The model will likely identify:
    # - 'University of California, Berkeley' as an ORG (Organization)
    # - 'United States' as a GPE (Geopolitical Entity)
    print(f"Entity: {ent.text}, Label: {ent.label_}")



Entity: The University of California, Berkeley, Label: ORG
Entity: the United States, Label: GPE


### 5.6 **Applications of Named Entity Recognition**



#### 5.6.1 Question Answering Systems
- **Example**: Using NER to extract the answer to a question.



In [ ]:
import spacy

# Load a spaCy pre-trained model for Named Entity Recognition (NER)
nlp = spacy.load("en_core_web_sm")

# Sentence containing relevant information for a question
# This sentence provides information about the person who founded SpaceX
text = "Elon Musk founded SpaceX in 2002."

# Question: "Who founded SpaceX?"
question = "Who founded SpaceX?"

# Processing the text to extract named entities
# The Doc object will contain information about the tokens and entities in the text
doc = nlp(text)

# Extracting the answer to the question based on entity labels
for ent in doc.ents:
    # If the entity is labeled as a PERSON and the word 'founded' appears in the sentence,
    # we assume that the person is the answer to the question
    if ent.label_ == "PERSON" and "founded" in text:
        # Print the entity text as the answer
        print(f"Answer: {ent.text}")



Answer: Elon Musk


#### 5.6.2 Resume Parsing
- **Example**: Extracting information such as name, skills, and education from resumes.


In [ ]:
import spacy

# Load a spaCy pre-trained model for Named Entity Recognition (NER)
nlp = spacy.load("en_core_web_sm")

# Sample resume text
resume_text = """
John Doe
Email: john.doe@email.com
Experience: Software Engineer at TechCorp from 2018 to 2022
Education: B.Sc. in Computer Science from Stanford University
Skills: Python, Machine Learning, Data Analysis
"""

# Processing the resume text with spaCy to create a Doc object
# The Doc object contains linguistic annotations, including named entities
doc = nlp(resume_text)

# Extracting named entities related to person names and organizations
for ent in doc.ents:
    # Recognizing the person's name, organizations, and educational institutions
    if ent.label_ in ["PERSON", "ORG"]:
        print(f"Entity: {ent.text}, Label: {ent.label_}")



Entity: John Doe, Label: PERSON
Entity: TechCorp, Label: ORG
Entity: Computer Science, Label: ORG
Entity: Stanford University, Label: ORG
Entity: Machine Learning, Label: PERSON
Entity: Data Analysis, Label: ORG


#### 5.6.3 Financial Analysis
- **Example**: Extracting financial entities from news articles or reports.


In [ ]:
import spacy

# Load a spaCy pre-trained model for Named Entity Recognition (NER)
nlp = spacy.load("en_core_web_sm")

# Financial report text
text = "In Q1 2022, Microsoft reported a revenue of $50 billion."

# Processing the text with spaCy to create a Doc object
# The Doc object contains linguistic annotations, including named entities
doc = nlp(text)

# Extracting monetary values, company names, and dates from the text
for ent in doc.ents:
    # Identifying entities labeled as ORG (Organization), MONEY, or DATE
    if ent.label_ in ["ORG", "MONEY", "DATE"]:
        print(f"Entity: {ent.text}, Label: {ent.label_}")

Entity: Q1 2022, Label: DATE
Entity: Microsoft, Label: ORG
Entity: $50 billion, Label: MONEY


#### 5.6.4 Healthcare and Biomedical Applications
- **Example**: Extracting drug names and conditions from clinical text.


In [ ]:

# Clinical document text
text = "The patient was prescribed Aspirin to manage cardiovascular issues."

# Processing the text
doc = nlp(text)

# Extracting named entities related to drugs and medical conditions
for ent in doc.ents:
    # Identifying drug names and medical conditions
    if ent.label_ in ["DRUG", "CONDITION"]:  # Assuming we have trained to detect these labels
        print(f"Entity: {ent.text}, Label: {ent.label_}")

# Note: The default spaCy model does not include medical labels, so custom training is required.


### 5.7 **Creative Observations in Named Entity Recognition**



#### 5.7.1 Transfer Learning for NER
- **Description**: Using pre-trained language models like BERT for entity recognition.
- **Example**: Fine-tuning BERT to extract entities in a specific domain.


In [ ]:
from transformers import pipeline

# Load a pre-trained model for Named Entity Recognition (NER) using the transformers library
# The model is fine-tuned on the CoNLL-03 dataset and is capable of recognizing common entity types
nlp = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

# Text containing named entities to analyze
text = "Tim Berners-Lee invented the World Wide Web while working at CERN."

# Using the pre-trained NER model to identify entities in the text
result = nlp(text)

# Displaying the extracted entities
for entity in result:
    # Each entity is represented as a dictionary with keys such as 'word', 'entity', 'score', etc.
    print(entity)



Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this 

{'entity': 'I-PER', 'score': 0.9997209, 'index': 1, 'word': 'Tim', 'start': 0, 'end': 3}
{'entity': 'I-PER', 'score': 0.99939823, 'index': 2, 'word': 'Bern', 'start': 4, 'end': 8}
{'entity': 'I-PER', 'score': 0.9889016, 'index': 3, 'word': '##ers', 'start': 8, 'end': 11}
{'entity': 'I-PER', 'score': 0.9910424, 'index': 4, 'word': '-', 'start': 11, 'end': 12}
{'entity': 'I-PER', 'score': 0.99914813, 'index': 5, 'word': 'Lee', 'start': 12, 'end': 15}
{'entity': 'I-MISC', 'score': 0.9989945, 'index': 8, 'word': 'World', 'start': 29, 'end': 34}
{'entity': 'I-MISC', 'score': 0.99818265, 'index': 9, 'word': 'Wide', 'start': 35, 'end': 39}
{'entity': 'I-MISC', 'score': 0.99867815, 'index': 10, 'word': 'Web', 'start': 40, 'end': 43}
{'entity': 'I-ORG', 'score': 0.99261415, 'index': 14, 'word': 'CE', 'start': 61, 'end': 63}
{'entity': 'I-ORG', 'score': 0.9871033, 'index': 15, 'word': '##R', 'start': 63, 'end': 64}
{'entity': 'I-ORG', 'score': 0.99020654, 'index': 16, 'word': '##N', 'start': 64,

#### 5.7.2 Active Learning for Efficient Labeling
- **Description**: Using active learning to select the most informative examples for labeling.
- **Example**: Selecting ambiguous sentences for human review to improve model training.


In [ ]:
# Sample texts with varying levels of ambiguity
texts = [
    "Barack Obama was the president of the United States.",
    "TechStar has launched a new AI product in Silicon Valley.",
    "The company Orange is expanding its market."
]

# Mock function to simulate active learning example selection
# The goal is to select the texts that may need human review due to ambiguity
def active_learning_selection(texts):
    selected_texts = []
    for text in texts:
        # Check if the text contains ambiguous terms
        # The terms "Orange" and "TechStar" are considered ambiguous or new, which may require human review
        if "Orange" in text or "TechStar" in text:
            selected_texts.append(text)
    return selected_texts

# Selecting the most informative examples for human review
# Ambiguous or unclear texts are selected for further labeling by human annotators
informative_texts = active_learning_selection(texts)
print("Texts selected for labeling:", informative_texts)


Texts selected for labeling: ['TechStar has launched a new AI product in Silicon Valley.', 'The company Orange is expanding its market.']



#### 5.7.3 NER for Cross-Lingual Applications
- **Description**: Using multilingual models to recognize named entities in different languages.
- **Example**: Fine-tuning a multilingual BERT model for recognizing entities in Spanish text.



In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

# Load a pre-trained multilingual BERT model for Named Entity Recognition (NER)
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Example Spanish text containing named entities
# This example uses the name "Lionel Messi" and geographical locations such as "Rosario, Argentina"
text = "Lionel Messi nació en Rosario, Argentina."

# Tokenize and encode the input text
# The tokenizer splits the text into subword tokens and converts them into input IDs that the model can understand
inputs = tokenizer(text, return_tensors="pt")

# Make predictions using the model
# The outputs contain logits (raw predictions) for each token
outputs = model(**inputs)

# Extract the logits from the model outputs
logits = outputs.logits

# Print the raw model outputs (logits)
# Note: Further processing is needed to convert these logits into human-readable entity labels
print(logits)

# Note: To extract meaningful entity labels, you would need to apply additional steps such as:
# 1. Identifying the most likely entity label for each token.
# 2. Converting the token predictions to readable text using the tokenizer.
# This involves using a softmax function and decoding the output.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[[-0.2017,  0.0464],
         [-0.2822,  0.1278],
         [-0.1109,  0.2427],
         [-0.1938,  0.2537],
         [-0.1815,  0.1103],
         [-0.0674, -0.1804],
         [-0.3432,  0.1501],
         [-0.0444,  0.1419],
         [-0.1419,  0.3257],
         [-0.1643,  0.2564]]], grad_fn=<ViewBackward0>)



#### 5.7.4 Integration with Knowledge Graphs
- **Description**: Linking extracted entities to nodes in a knowledge graph to enhance understanding.
- **Example**: Creating RDF triples for linking extracted entities.


In [ ]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.0 MB/s eta 0:00:00


In [ ]:
from rdflib import Graph, URIRef

# Create a new RDF graph
g = Graph()

# Define URIs for entities and relationships
# Defining unique identifiers for Elon Musk, the founded relationship, and SpaceX
elon_musk = URIRef("http://example.org/ElonMusk")
founded = URIRef("http://example.org/founded")
spacex = URIRef("http://example.org/SpaceX")

# Add triples to the graph to represent the relationship
# Adding a triple to state that "Elon Musk founded SpaceX"
g.add((elon_musk, founded, spacex))

# Print all triples in the graph
# Iterating through the graph to print each subject-predicate-object (SPO) triple
for s, p, o in g:
    print(s, p, o)



http://example.org/ElonMusk http://example.org/founded http://example.org/SpaceX
